In [55]:
import numpy as np
import pandas as pd

# read node data in coordinate (x,y) format
nodes = pd.read_csv('real_demand_bak2.csv', index_col = 'node')
nodes.rename(columns={"distance to depot":'d0'}, inplace = True)
node_number = len(nodes.index) - 1
nodes.head()


# read pairwise distance
pw = pd.read_csv('real_pairwise.csv', index_col = 'Unnamed: 0')
pw.index.rename('',inplace = True)
pw


# calculate savings for each link
savings = dict()
for r in pw.index:
    for c in pw.columns:
        if int(c) != int(r):            
            a = max(int(r), int(c))
            b = min(int(r), int(c))
            key = '(' + str(a) + ',' + str(b) + ')'
            #print(nodes['d0'][int(r)])
            savings[key] = nodes['d0'][int(r)] + nodes['d0'][int(c)] - pw[c][r]

# put savings in a pandas dataframe, and sort by descending
sv = pd.DataFrame.from_dict(savings, orient = 'index')
sv.rename(columns = {0:'saving'}, inplace = True)
sv.sort_values(by = ['saving'], ascending = False, inplace = True)
sv.head(15)




,saving
"(120,116)",376
"(119,116)",375
"(120,119)",374
"(116,112)",374
"(122,121)",370
"(120,112)",370
"(119,112)",369
"(121,116)",368
"(122,116)",367
"(122,120)",366


In [56]:
# convert link string to link list to handle saving's key, i.e. str(10, 6) to (10, 6)
def get_node(link):
    link = link[1:]
    link = link[:-1]
    nodes = link.split(',')
    return [int(nodes[0]), int(nodes[1])]



# determine if a node is interior to a route
def interior(node, route):
    try:
        i = route.index(node)
        # adjacent to depot, not interior
        if i == 0 or i == (len(route) - 1):
            label = False
        else:
            label = True
    except:
        label = False
    
    return label



def merge(route0, route1, link):
    if route0.index(link[0]) != (len(route0) - 1):
        route0.reverse()
    
    if route1.index(link[1]) != 0:
        route1.reverse()
        
    return route0 + route1


# sum up to obtain the total passengers belonging to a route
def sum_cap(route):
    sum_cap = 0
    for node in route:
        sum_cap += nodes.demand[node]
    return sum_cap



# determine 4 things:
# 1. if the link in any route in routes -> determined by if count_in > 0
# 2. if yes, which node is in the route -> returned to node_sel
# 3. if yes, which route is the node belongs to -> returned to route id: i_route
# 4. are both of the nodes in the same route? -> overlap = 1, yes; otherwise, no
def which_route(link, routes):
    # assume nodes are not in any route
    node_sel = list()
    i_route = [-1, -1]
    count_in = 0
    
    for route in routes:
        for node in link:
            try:
                route.index(node)
                i_route[count_in] = routes.index(route)
                node_sel.append(node)
                count_in += 1
            except:
                a = 1
                
    if i_route[0] == i_route[1]:
        overlap = 1
    else:
        overlap = 0
        
    return node_sel, count_in, i_route, overlap

print('nano')





nano


In [57]:
# create empty routes
#routes = list()

# if there is any remaining customer to be served
routes = list()
remaining = True

# determine capacity of the vehicle
cap = 3200

# record steps
step = 0

# get a list of nodes, excluding the depot
node_list = list(nodes.index)
node_list.remove(0)
print(node_list)
# run through each link in the saving list
for link in sv.index:
    step += 1
    if remaining:

        print('step ', step, ':')

        link = get_node(link)
        node_sel, num_in, i_route, overlap = which_route(link, routes)

        # condition a. Either, neither i nor j have already been assigned to a route, 
        # ...in which case a new route is initiated including both i and j.
        if num_in == 0:
            if sum_cap(link) <= cap:
                routes.append(link)
                node_list.remove(link[0])
                node_list.remove(link[1])
                print('\t','Link ', link, ' fulfills criteria a), so it is created as a new route')
            else:
                print('\t','Though Link ', link, ' fulfills criteria a), it exceeds maximum load, so skip this link.')

        # condition b. Or, exactly one of the two nodes (i or j) has already been included 
        # ...in an existing route and that point is not interior to that route 
        # ...(a point is interior to a route if it is not adjacent to the depot D in the order of traversal of nodes), 
        # ...in which case the link (i, j) is added to that same route.    
        elif num_in == 1:
            n_sel = node_sel[0]
            i_rt = i_route[0]
            position = routes[i_rt].index(n_sel)
            link_temp = link.copy()
            link_temp.remove(n_sel)
            node = link_temp[0]

            cond1 = (not interior(n_sel, routes[i_rt]))
            cond2 = (sum_cap(routes[i_rt] + [node]) <= cap)

            if cond1:
                if cond2:
                    print('\t','Link ', link, ' fulfills criteria b), so a new node is added to route ', routes[i_rt], '.')
                    if position == 0:
                        routes[i_rt].insert(0, node)
                    else:
                        routes[i_rt].append(node)
                    node_list.remove(node)
                else:
                    print('\t','Though Link ', link, ' fulfills criteria b), it exceeds maximum load, so skip this link.')
                    continue
            else:
                print('\t','For Link ', link, ', node ', n_sel, ' is interior to route ', routes[i_rt], ', so skip this link')
                continue

        # condition c. Or, both i and j have already been included in two different existing routes 
        # ...and neither point is interior to its route, in which case the two routes are merged.        
        else:
            if overlap == 0:
                cond1 = (not interior(node_sel[0], routes[i_route[0]]))
                cond2 = (not interior(node_sel[1], routes[i_route[1]]))
                cond3 = (sum_cap(routes[i_route[0]] + routes[i_route[1]]) <= cap)

                if cond1 and cond2:
                    if cond3:
                        route_temp = merge(routes[i_route[0]], routes[i_route[1]], node_sel)
                        temp1 = routes[i_route[0]]
                        temp2 = routes[i_route[1]]
                        routes.remove(temp1)
                        routes.remove(temp2)
                        routes.append(route_temp)
                        print('nano1',node_list)
                        print('nano1',link[0],link[1])
                        if link[0] in node_list:
                            node_list.remove(link[0])
                        if link[1] in node_list:
                            node_list.remove(link[0])
                        #node_list.remove(link[0])
                        #node_list.remove(link[1])
                        print('testing this condition')
                        print('\t','Link ', link, ' fulfills criteria c), so route ', temp1, ' and route ', temp2, ' are merged')
                    else:
                        print('\t','Though Link ', link, ' fulfills criteria c), it exceeds maximum load, so skip this link.')
                        continue
                else:
                    print('\t','For link ', link, ', Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link')
                    continue
            else:
                print('\t','Link ', link, ' is already included in the routes')
                continue

        for route in routes: 
            print('\t','route: ', route, ' with load ', sum_cap(route))
    else:
        print('-------')
        print('All nodes are included in the routes, algorithm closed')
        break

    remaining = bool(len(node_list) > 0)

# check if any node is left, assign to a unique route
for node_o in node_list:
    routes.append([node_o])

# add depot to the routes
for route in routes:
    route.insert(0,0)
    route.append(0)

print('------')
print('Routes found are: ')
routes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]
step  1 :
	 Link  [120, 116]  fulfills criteria a), so it is created as a new route
	 route:  [120, 116]  with load  0
step  2 :
	 Link  [119, 116]  fulfills criteria b), so a new node is added to route  [120, 116] .
	 route:  [120, 116, 119]  with load  0
step  3 :
	 Link  [120, 119]  is already included in the routes
step  4 :
	 For Link  [116, 112] , node  116  is interior to route  [120, 116, 119] , so skip this link
step  5 :
	 Link  [122, 121]  fulfills criteria a), so it is created as 

	 For Link  [105, 104] , node  104  is interior to route  [109, 108, 117, 122, 121, 112, 120, 116, 119, 113, 115, 114, 118, 110, 111, 107, 101, 104, 103] , so skip this link
step  167 :
	 For Link  [116, 106] , node  116  is interior to route  [109, 108, 117, 122, 121, 112, 120, 116, 119, 113, 115, 114, 118, 110, 111, 107, 101, 104, 103] , so skip this link
step  168 :
	 Link  [105, 103]  fulfills criteria b), so a new node is added to route  [109, 108, 117, 122, 121, 112, 120, 116, 119, 113, 115, 114, 118, 110, 111, 107, 101, 104, 103] .
	 route:  [109, 108, 117, 122, 121, 112, 120, 116, 119, 113, 115, 114, 118, 110, 111, 107, 101, 104, 103, 105]  with load  457
step  169 :
	 Link  [113, 103]  is already included in the routes
step  170 :
	 Link  [118, 103]  is already included in the routes
step  171 :
	 Link  [108, 104]  is already included in the routes
step  172 :
	 For Link  [119, 106] , node  119  is interior to route  [109, 108, 117, 122, 121, 112, 120, 116, 119, 113, 115, 114,

step  356 :
	 Link  [113, 78]  is already included in the routes
step  357 :
	 Link  [110, 78]  is already included in the routes
step  358 :
	 For link  [108, 82] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  359 :
	 For link  [109, 82] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  360 :
	 For link  [117, 82] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  361 :
	 Link  [99, 72]  is already included in the routes
step  362 :
	 Link  [77, 72]  is already included in the routes
step  363 :
	 Link  [100, 79]  is already included in the routes
step  364 :
	 Link  [101, 78]  is already included in the routes
step  365 :
	 Link  [100, 78]  is already included in the routes
step  366 :
	 Link  [116, 72]  is already included in the routes
step  367 :
	 Link  [120, 72]

	 Link  [91, 86]  is already included in the routes
step  491 :
	 Link  [90, 86]  is already included in the routes
step  492 :
	 Link  [88, 86]  is already included in the routes
step  493 :
	 For link  [117, 85] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  494 :
	 For link  [109, 85] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  495 :
	 For link  [108, 85] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  496 :
	 For link  [106, 85] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  497 :
	 Link  [75, 73]  fulfills criteria a), so it is created as a new route
	 route:  [96, 95, 97]  with load  776
	 route:  [83, 84, 74, 82, 72, 79, 78, 77, 99, 106, 109, 108, 117, 122, 121, 112, 120, 116, 1

	 For link  [103, 87] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  639 :
	 For link  [104, 87] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  640 :
	 For link  [114, 87] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  641 :
	 For link  [121, 88] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  642 :
	 For link  [122, 88] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  643 :
	 For link  [99, 90] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  644 :
	 For link  [115, 92] , Two nodes are found in two different routes, but not all the nodes fulf

...

	 For Link  [117, 13] , node  117  is interior to route  [18, 4, 76, 75, 73, 98, 100, 102, 105, 103, 104, 101, 107, 111, 110, 118, 114, 115, 113, 119, 116, 120, 112, 121, 122, 117, 108, 109, 106, 99, 77, 78, 79, 72, 82, 74, 84, 83, 80, 2, 1, 62, 3, 19, 14, 15] , so skip this link
step  5649 :
	 For Link  [106, 13] , node  106  is interior to route  [18, 4, 76, 75, 73, 98, 100, 102, 105, 103, 104, 101, 107, 111, 110, 118, 114, 115, 113, 119, 116, 120, 112, 121, 122, 117, 108, 109, 106, 99, 77, 78, 79, 72, 82, 74, 84, 83, 80, 2, 1, 62, 3, 19, 14, 15] , so skip this link
step  5650 :
	 For Link  [118, 13] , node  118  is interior to route  [18, 4, 76, 75, 73, 98, 100, 102, 105, 103, 104, 101, 107, 111, 110, 118, 114, 115, 113, 119, 116, 120, 112, 121, 122, 117, 108, 109, 106, 99, 77, 78, 79, 72, 82, 74, 84, 83, 80, 2, 1, 62, 3, 19, 14, 15] , so skip this link
step  5651 :
	 For link  [62, 28] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement

	 For link  [95, 70] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5738 :
	 For link  [62, 44] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5739 :
	 For link  [49, 13] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5740 :
	 For link  [52, 13] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5741 :
	 For link  [108, 22] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5742 :
	 For link  [75, 13] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5743 :
	 For link  [74, 13] , Two nodes are found in two different routes, but not all the nodes ful

	 For link  [109, 23] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5839 :
	 For link  [72, 23] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5840 :
	 For link  [110, 23] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5841 :
	 For link  [111, 23] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5842 :
	 For link  [112, 23] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5843 :
	 For link  [57, 23] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5844 :
	 For link  [56, 23] , Two nodes are found in two different routes, but not all the nodes 

	 For link  [60, 2] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5937 :
	 For link  [38, 9] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5938 :
	 For link  [22, 2] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5939 :
	 For link  [100, 69] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5940 :
	 For link  [114, 69] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5941 :
	 Link  [59, 9]  is already included in the routes
step  5942 :
	 For link  [41, 9] , Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link
step  5943 :
	 For link  [29, 9] , Two nodes 

[[0,
  25,
  37,
  36,
  42,
  33,
  34,
  48,
  32,
  49,
  8,
  11,
  10,
  6,
  7,
  5,
  93,
  94,
  81,
  89,
  92,
  86,
  85,
  88,
  90,
  91,
  87,
  97,
  95,
  96,
  53,
  54,
  51,
  56,
  55,
  57,
  52,
  43,
  44,
  39,
  40,
  28,
  30,
  29,
  24,
  26,
  41,
  27,
  35,
  38,
  20,
  12,
  17,
  0],
 [0,
  13,
  45,
  47,
  46,
  50,
  9,
  16,
  58,
  60,
  61,
  59,
  64,
  71,
  63,
  65,
  66,
  67,
  68,
  70,
  69,
  23,
  22,
  21,
  31,
  0],
 [0,
  18,
  4,
  76,
  75,
  73,
  98,
  100,
  102,
  105,
  103,
  104,
  101,
  107,
  111,
  110,
  118,
  114,
  115,
  113,
  119,
  116,
  120,
  112,
  121,
  122,
  117,
  108,
  109,
  106,
  99,
  77,
  78,
  79,
  72,
  82,
  74,
  84,
  83,
  80,
  2,
  1,
  62,
  3,
  19,
  14,
  15,
  0]]